### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
import os
import sys

import numpy as np
import numba as nb
import pandas as pd
import multiprocessing as mp

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [2]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Dynamically Re-Load all Package Modules on Execution

In [3]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

#### Set File Path Constants

In [4]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [5]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Load Items Data

In [6]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Load Ratings Data

In [8]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

In [9]:
ratings_explicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,196,242,3,1997-12-04 15:55:49,0
1,186,302,3,1998-04-04 19:22:22,0
2,22,377,1,1997-11-07 07:18:36,0
3,244,51,2,1997-11-27 05:02:03,0
4,166,346,1,1998-02-02 05:33:16,0


#### Generate Implicit Feedback Ratings Data

In [10]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [12]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create (Training, Validation) Interactions for Evaluation

In [13]:
print(ratings_implicit.shape)
ratings_implicit.head()

(54194, 5)


,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


In [15]:
test_pct = 0.25
drop_users = (2, 4)

train_mask = (ratings_implicit.index / ratings_implicit.shape[0]) <  (1 - test_pct)
valid_mask = (ratings_implicit.index / ratings_implicit.shape[0]) >= (1 - test_pct)

interactions_total = ratings_implicit[['user_id', 'item_id']]
interactions_train = ratings_implicit[train_mask][['user_id', 'item_id']]
interactions_valid = ratings_implicit[valid_mask][['user_id', 'item_id']]

# drop some users from the training data to test cold-start functionality
interactions_train = interactions_train[~interactions_train.user_id.isin(drop_users)]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

print("total shape: {}".format(interactions_total.shape))
print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))
print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

total shape: (100000, 2)
train shape: (74924, 2)
valid shape: (25000, 2)
train users: 941
valid users: 934
cold-start users: {2, 4}


In [16]:
iteraction_users = interactions_train.user_id.unique()
iteraction_items = interactions_train.item_id.unique()

#### Create User/Item Features Data for Testing

In [17]:
user_features = users_df[users_df.user_id.isin(iteraction_users)]
print(user_features.shape, users_df.shape)
user_features.head()

(941, 27) (943, 27)


,user_id,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,6,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
item_features = items_df[items_df.item_id.isin(iteraction_items)]
print(item_features.shape, items_df.shape)
item_features.head()

(1643, 19) (1682, 19)


,item_id,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


### Test Out Core Package Functionality

#### Initialize the Model

In [19]:
model = RankFM(factors=10, learning_rate=0.1, regularization=0.01, sigma=0.1)
model

#### Initialize and Inspect Internal Data

In [20]:
%%prun -l 10
model._init_all(interactions_train, user_features, item_features)

/Users/ericlundquist/anaconda3/lib/python3.7/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value


         1245857 function calls (1184769 primitive calls) in 0.944 seconds

   Ordered by: internal time
   List reduced from 3280 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3379    0.132    0.000    0.133    0.000 ffi.py:111(__call__)
   246303    0.035    0.000    0.053    0.000 {built-in method builtins.isinstance}
31912/6043    0.027    0.000    0.039    0.000 ir.py:313(_rec_list_vars)
       67    0.016    0.000    0.016    0.000 {method 'read' of '_io.BufferedReader' objects}
       71    0.014    0.000    0.014    0.000 {built-in method io.open}
      404    0.014    0.000    0.014    0.000 {built-in method posix.stat}
       45    0.013    0.000    0.013    0.000 {method 'read' of '_io.FileIO' objects}
17132/8842    0.012    0.000    0.037    0.000 {method 'format' of 'str' objects}
  962/961    0.010    0.000    0.056    0.000 base.py:253(__new__)
2946/1056    0.009    0.000    0.033    0.000 {built-in method num

In [21]:
print(len(model.user_id), len(model.user_idx))
print(len(model.item_id), len(model.item_idx))

941 941
1643 1643


In [22]:
print(model.interactions.shape)
print(model.x_uf.shape)
print(model.x_if.shape)

(74924, 2)
(941, 26)
(1643, 18)


In [23]:
print("item weights: {}".format(model.w_i.shape))
print("item feature weights: {}".format(model.w_if.shape))
print("user factors: {}".format(model.v_u.shape))
print("item factors: {}".format(model.v_i.shape))
print("user feature factors: {}".format(model.v_uf.shape))
print("item feature factors: {}".format(model.v_if.shape))

item weights: (1643,)
item feature weights: (18,)
user factors: (941, 10)
item factors: (1643, 10)
user feature factors: (26, 10)
item feature factors: (18, 10)


In [24]:
model.interactions.dtype

dtype('int32')

In [25]:
model.user_items_nb[10]

array([  0,   1,   3,   4,   6,  10,  11,  12,  13,  16,  20,  21,  22,
        23,  24,  26,  28,  31,  32,  36,  37,  38,  39,  41,  44,  47,
        48,  49,  50,  52,  55,  57,  58,  60,  61,  63,  65,  66,  67,
        68,  69,  70,  71,  72,  77,  78,  81,  82,  85,  86,  87,  88,
        89,  90,  91,  93,  94,  95,  96,  97,  98,  99, 108, 109, 110,
       116, 117, 120, 123, 126, 127, 136, 137, 143, 144, 146, 149, 151,
       152, 153, 154, 156, 157, 159, 160, 162, 163, 166, 167, 169, 171,
       172, 173, 174, 176, 177, 178, 179, 180, 182, 183, 184, 185, 189,
       190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 204,
       207, 209, 210, 211, 214, 215, 216, 218, 221, 222, 223, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 236, 237, 238, 240, 242,
       257, 260, 261, 262, 263, 264, 267, 268, 269, 270, 271, 272, 273,
       274, 275, 278, 280, 285, 286, 287, 288, 289, 291, 293, 298, 299,
       300, 301, 302, 304, 305, 306, 307, 309, 310, 311, 312, 31

In [26]:
uf_checker = pd.DataFrame(model.x_uf, columns=user_features.columns[1:])
uf_checker.head()

,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,occupation__engineer,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
user_features[user_features.user_id.isin(pd.Series([0,1,2,3,4]).map(model.index_to_user))]

,user_id,agegroup__0,agegroup__1,agegroup__2,gender__F,gender__M,occupation__administrator,occupation__artist,occupation__doctor,occupation__educator,...,occupation__marketing,occupation__none,occupation__other,occupation__programmer,occupation__retired,occupation__salesman,occupation__scientist,occupation__student,occupation__technician,occupation__writer
0,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,6,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
if_checker = pd.DataFrame(model.x_if, columns=item_features.columns[1:])
if_checker.head()

,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
item_features[item_features.item_id.isin(pd.Series([0,1,2,3,4]).map(model.index_to_item))]

,item_id,genre__action,genre__adventure,genre__animation,genre__childrens,genre__comedy,genre__crime,genre__documentary,genre__drama,genre__fantasy,genre__film_noir,genre__horror,genre__musical,genre__mystery,genre__romance,genre__scifi,genre__thriller,genre__war,genre__western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [30]:
model = RankFM(factors=10, learning_rate=0.10, regularization=0.01, sigma=0.1)
model

#### Fit the Model on the Training Data and Profile Computational Performance

In [31]:
%%time

# model.fit(interactions_train, item_features=item_features, epochs=50, verbose=True)
model.fit(interactions_train, epochs=10, verbose=True)

/Users/ericlundquist/Repos/rankfm/rankfm/numba_methods.py:114: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, C))
  d_v_uf[p, f] = (x_uf[u][p]) * (v_i[i][f] - v_i[j][f] + np.dot(v_if.T[f], x_if[i] - x_if[j]))
/Users/ericlundquist/Repos/rankfm/rankfm/numba_methods.py:119: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, C))
  d_v_if[q, f] = (x_if[i][q] - x_if[j][q]) * (v_u[u][f] + np.dot(v_uf.T[f], x_uf[u]))



training epoch: 0
log likelihood: -33656.06

training epoch: 1
log likelihood: -28473.84

training epoch: 2
log likelihood: -25823.52

training epoch: 3
log likelihood: -22968.66

training epoch: 4
log likelihood: -21033.07

training epoch: 5
log likelihood: -19904.01

training epoch: 6
log likelihood: -18817.76

training epoch: 7
log likelihood: -18035.94

training epoch: 8
log likelihood: -17562.33

training epoch: 9
log likelihood: -17314.97
CPU times: user 9.25 s, sys: 85.8 ms, total: 9.33 s
Wall time: 9.36 s


In [32]:
# model.fit_partial(interactions_train, epochs=5, verbose=True)

#### Generate Model Scores

In [33]:
interactions_total.shape

(100000, 2)

In [34]:
%%time
scores = model.predict(interactions_total, cold_start='nan') 

CPU times: user 434 ms, sys: 6.3 ms, total: 440 ms
Wall time: 441 ms


#### Generate TopN Recommendations

In [35]:
all_users = pd.Series(interactions_total.user_id.unique())
print(all_users.shape)
all_users.head(10)

(943,)


0    196
1    186
2     22
3    244
4    166
5    298
6    115
7    253
8    305
9      6
dtype: int64

In [36]:
%%time
top_n = model.recommend_for_users(all_users, n_items=10, filter_previous=False, cold_start='nan')

CPU times: user 2.63 s, sys: 23 ms, total: 2.65 s
Wall time: 2.26 s


In [37]:
top_n.head(10)

,0,1,2,3,4,5,6,7,8,9
196,50.0,25.0,100.0,238.0,111.0,174.0,275.0,181.0,1.0,70.0
186,294.0,258.0,678.0,748.0,288.0,117.0,259.0,331.0,546.0,326.0
22,168.0,204.0,121.0,229.0,554.0,94.0,230.0,210.0,195.0,181.0
244,121.0,25.0,282.0,763.0,405.0,204.0,174.0,111.0,69.0,50.0
166,258.0,690.0,307.0,328.0,322.0,302.0,289.0,748.0,300.0,271.0
298,194.0,174.0,132.0,69.0,50.0,98.0,197.0,121.0,127.0,204.0
115,50.0,174.0,98.0,127.0,121.0,276.0,12.0,475.0,508.0,89.0
253,174.0,98.0,50.0,194.0,132.0,69.0,318.0,127.0,357.0,195.0
305,174.0,50.0,98.0,238.0,179.0,89.0,357.0,56.0,479.0,582.0
6,194.0,174.0,50.0,275.0,483.0,479.0,134.0,197.0,98.0,357.0


#### Evaluate Model Performance on the Validation Data

In [38]:
interactions_valid.head()

,user_id,item_id
75000,498,317
75001,840,492
75002,406,282
75003,221,847
75004,757,98


In [39]:
k = 10

In [40]:
%%time
model_pre = round(precision_at_k(model, interactions_valid, k=k), 3)
model_rec = round(recall_at_k(model, interactions_valid, k=k), 3)

CPU times: user 2.4 s, sys: 17.8 ms, total: 2.42 s
Wall time: 761 ms


In [41]:
print("model precision: {} model recall: {}".format(model_pre, model_rec))

model precision: 0.162 model recall: 0.092


#### Evaluate Pure Popularity Baseline

In [42]:
most_popular = interactions_train.groupby('item_id').size().sort_values(ascending=False)[:k]
most_popular

item_id
50     435
181    386
258    383
288    372
100    371
294    353
286    352
1      339
121    309
174    309
dtype: int64

In [43]:
test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_recommends = model.recommend_for_users(list(test_user_items.keys()), cold_start='drop')
test_user_items = {key: val for key, val in test_user_items.items() if key in test_recommends.index}

base_pre = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()]), 3)
base_rec = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()]), 3)

print("number of test users: {}".format(len(test_user_items)))
print("baseline precision: {} baseline recall: {}".format(base_pre, base_rec))

number of test users: 932
baseline precision: 0.132 baseline recall: 0.08


#### Generate Model Scores for Test User/Items

In [51]:
mask = (interactions_valid.user_id.isin(interactions_train.user_id)) & (interactions_valid.item_id.isin(interactions_train.item_id))
print(interactions_valid.shape)
print(interactions_valid[mask].shape)

(13548, 2)
(13469, 2)


In [52]:
scores_miss = model.predict(interactions_valid, cold_start='nan')
scores_drop = model.predict(interactions_valid, cold_start='drop')

print(scores_miss.shape)
print(scores_drop.shape)

(13548,)
(13469,)


In [53]:
print(scores_miss.mean(), scores_miss.std(), scores_miss.min(), scores_miss.max())
print(scores_drop.mean(), scores_drop.std(), scores_drop.min(), scores_drop.max())

nan nan nan nan
1.5339588 1.1210613 -3.1286857 4.845548


#### Spot-Check Some User Recommendations

In [54]:
len(train_users), len(valid_users), cold_start_users

(941, 905, {2, 4})

In [55]:
recommendations = model.recommend_for_users(valid_users, n_items=10, filter_previous=True, cold_start='nan')
print(recommendations.shape)
recommendations.head(10)

(905, 10)


,0,1,2,3,4,5,6,7,8,9
1,83.0,172.0,652.0,179.0,433.0,428.0,425.0,474.0,216.0,475.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,302.0,272.0,333.0,270.0,313.0,292.0,346.0,305.0,288.0,316.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,174.0,68.0,96.0,161.0,82.0,156.0,181.0,568.0,83.0,175.0
6,603.0,286.0,172.0,657.0,496.0,179.0,582.0,151.0,604.0,607.0
7,357.0,23.0,318.0,419.0,604.0,197.0,525.0,82.0,173.0,69.0
8,265.0,234.0,385.0,96.0,98.0,12.0,7.0,168.0,520.0,199.0
9,100.0,127.0,181.0,275.0,150.0,258.0,315.0,269.0,283.0,302.0
10,180.0,318.0,98.0,474.0,194.0,172.0,528.0,23.0,174.0,89.0


In [56]:
recommendations[recommendations.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
user_id = 5
user_recs = recommendations.loc[user_id]
user_item_names = item_names[item_names.item_id.isin(user_recs)].set_index('item_id').loc[user_recs]
user_item_names

,item_name
item_id,
174.0,Raiders of the Lost Ark (1981)
68.0,"Crow, The (1994)"
96.0,Terminator 2: Judgment Day (1991)
161.0,Top Gun (1986)
82.0,Jurassic Park (1993)
156.0,Reservoir Dogs (1992)
181.0,Return of the Jedi (1983)
568.0,Speed (1994)
83.0,Much Ado About Nothing (1993)


#### Look at Similar Movies for a Few Recommended Movies

In [63]:
most_similar_items = model.similar_items(181.0)
most_similar_names = item_names.set_index('item_id').loc[most_similar_items]
most_similar_names

,item_name
item_id,
172,"Empire Strikes Back, The (1980)"
176,Aliens (1986)
144,Die Hard (1988)
265,"Hunt for Red October, The (1990)"
161,Top Gun (1986)
385,True Lies (1994)
174,Raiders of the Lost Ark (1981)
228,Star Trek: The Wrath of Khan (1982)
87,Searching for Bobby Fischer (1993)


### Look at the Movies with the Highest/Lowest Model Weights

In [53]:
item_weights = pd.DataFrame({'item_idx': np.arange(len(model.w_i)), 'w_item': model.w_i})
item_weights['item_id'] = item_weights['item_idx'].map(model.index_to_item)
item_weights = pd.merge(item_weights, item_names, on='item_id').sort_values('w_item', ascending=False)
item_weights = item_weights[['item_id', 'item_idx', 'item_name', 'w_item']]

#### Best Movies Ever?

In [54]:
item_weights.head(10)

,item_id,item_idx,item_name,w_item
49,50,49,Star Wars (1977),2.312693
99,100,99,Fargo (1996),2.213715
124,127,124,"Godfather, The (1972)",2.178087
310,313,310,Titanic (1997),2.112038
255,258,255,Contact (1997),2.064679
283,286,283,"English Patient, The (1996)",2.050860
178,181,178,Return of the Jedi (1983),1.937805
97,98,97,"Silence of the Lambs, The (1991)",1.867586
21,22,21,Braveheart (1995),1.824096
171,174,171,Raiders of the Lost Ark (1981),1.785299


#### Worst Movies Ever?

In [55]:
item_weights.tail(10)

,item_id,item_idx,item_name,w_item
1295,1413,1295,Street Fighter (1994),-1.712822
536,548,536,"NeverEnding Story III, The (1994)",-1.715896
1169,1227,1169,"Awfully Big Adventure, An (1995)",-1.716949
442,453,442,Jaws 3-D (1983),-1.719923
1244,1324,1244,Loaded (1994),-1.728409
958,996,958,"Big Green, The (1995)",-1.729017
1383,1554,1383,Safe Passage (1994),-1.731241
1323,1458,1323,"Damsel in Distress, A (1937)",-1.744731
1261,1370,1261,I Can't Sleep (J'ai pas sommeil) (1994),-1.746733
1321,1452,1321,Lady of Burlesque (1943),-1.780282


# Start Sandbox Code

#### implement binary search